In [92]:
import utils

import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_similarity_score

import keras
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, InputLayer
from keras.optimizers import RMSprop, Adam, SGD

import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import scipy
from tqdm import tqdm

from os.path import join, basename, exists
from os import makedirs, listdir
import json

In [2]:
def preproc_target_train(data, reduce_classes=False):
    diagnoses = data['Код_диагноза'].copy()
    
    if reduce_classes:
        pop_diagnoses = set(utils.get_most_popular_diagnoses(diagnoses, percent=.80))
        most_pop_diagnose = scipy.stats.mode(diagnoses)[0][0]
    else:
        pop_diagnoses = set(diagnoses)
        most_pop_diagnose = scipy.stats.mode(diagnoses)[0][0]
    
    diagnoses = diagnoses.apply(
        lambda diag: diag if diag in pop_diagnoses else most_pop_diagnose
    )
    
    return diagnoses, pop_diagnoses, most_pop_diagnose

def preproc_target_test(data, pop_diagnoses, most_pop_diagnose):
    diagnoses = data['Код_диагноза'].copy()
    
    diagnoses = diagnoses.apply(
        lambda diag: diag if diag in pop_diagnoses else most_pop_diagnose
    )
    
    return diagnoses, pop_diagnoses, most_pop_diagnose

def join_topics(data, topics):
    data = data.copy()
    
    topics_df = pd.DataFrame(dict(zip(
        ['topic' + str(i) for i in range(topics.shape[1])],
        [topics[:, i] for i in range(topics.shape[1])])))
    topics_df['Id_Записи'] = data['Id_Записи']

    data = data.join(topics_df, on='Id_Записи', rsuffix='_topics', how='outer')
    data = data.drop(columns=['Id_Записи_topics'])
    
    return data

In [74]:
class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

class DoctorsPopularityTransformator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        doctors = x.fillna('sss')
        doctors_voc, counts = np.unique(doctors, return_counts=True)
        self.pop_doctor = doctors_voc[np.argsort(counts)[::-1][0]]
        
        return self

    def transform(self, x):
        x = x.fillna('sss')
        x[x == 'sss'] = self.pop_doctor
        
        return x
    
class GenderTransformator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):        
        x = x.copy()
        x[x == 1] = 0
        x[x == 2] = 1
        x = x.astype(np.float32)
        
        return np.expand_dims(x, axis=1)
    
class AgeTransformator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        x = x.astype(np.float32)
        return np.expand_dims(x, axis=1)
    
class TopicsTransformator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        return x.astype(np.float32)

class ClinicTransformator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        clinics, counts = np.unique(x, return_counts=True)
        self.clinics_to_idx = dict(zip(clinics, range(len(clinics))))
        
        self.most_pop_clinic_idx = np.argmax(counts)
        
        return self
    
    def transform(self, x):
        x_vec = np.zeros((x.shape[0], len(self.clinics_to_idx)), dtype=np.float32)
        for i, clinic in enumerate(x):
            if clinic in self.clinics_to_idx:
                x_vec[i, self.clinics_to_idx[clinic]] = 1.
            else:
                x_vec[i, self.most_pop_clinic_idx] = 1.
        
        return x_vec
    
class RepeatsTransformator(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        x = x.astype(np.float32)
        return np.expand_dims(x, axis=1)

In [75]:
class NNModel(BaseEstimator, TransformerMixin):
    def __init__(self, batch_size, epochs, tb_log_dir, model_path, validation_split=0.3):
        self.batch_size = batch_size
        self.epochs = epochs
        self.tb_log_dir = tb_log_dir
        self.model_path = model_path
        self.validation_split = validation_split
    
    def fit(self, x, y=None):
        self.classes = np.unique(y)
        self.classes_voc = dict(zip(self.classes, range(self.classes.shape[0])))
        self.voc_classes = dict(zip(range(self.classes.shape[0]), self.classes))
        y_proc = np.zeros((y.shape[0], self.classes.shape[0]), dtype=np.float32)
        for i, yc in enumerate(y):
            y_proc[i, self.classes_voc[yc]] = 1.
        
        self.model = Sequential([
            InputLayer(input_shape=(x.shape[1],)),
            Dense(1024, activation='sigmoid'),
#             Dense(2048, activation='sigmoid'),
#             Dense(2048, activation='sigmoid'),
#             Dense(4096, activation='sigmoid'),
#             Dense(4096, activation='sigmoid'),
            Dense(self.classes.shape[0], activation='softmax')
        ])
        
        optim = RMSprop(lr=1e-4, decay=1e-6)
        self.model.compile(optimizer=optim, loss='categorical_crossentropy', metrics=['accuracy'])
        
        self.model.fit(x, y_proc,
                  batch_size=self.batch_size, epochs=self.epochs,
                  callbacks=[
                      TensorBoard(log_dir=self.tb_log_dir, batch_size=self.batch_size),
                      ModelCheckpoint(filepath=self.model_path, monitor='acc', period=5)
                  ],
                  validation_split=self.validation_split)
        
        return self
        
    def transform(self, x):
        pred = self.model.predict(x)
        max_idxs = np.argmax(pred, axis=1)
        
        return np.array(list(map(lambda max_idx: self.voc_classes[max_idx], max_idxs)))

In [93]:
def create_pipeline(experiment_dir, model_fname_prefix, no_validation=False, validation_split=0.3):
    model_name = model_fname_prefix + str(utils.get_next_model_id(experiment_dir))
    tb_log_dir = join(experiment_dir, 'log', model_name)
    models_dir = join(experiment_dir, 'models')

    if no_validation:
        model_path = utils.get_model_fname_pattern(models_dir, model_name, no_validation=True)
        validation_split = 0.0
    else:
        model_path = utils.get_model_fname_pattern(models_dir, model_name, no_validation=False)

    pipe = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('complaints_pipe', Pipeline([
                    ('complaint_selector', ItemSelector(key='Жалобы (unigramm)')),
                    ('tfidf', TfidfVectorizer(ngram_range=(1,1), min_df=1, stop_words=stopwords.words('russian')))
                ])),
                ('complaints_n_pipe', Pipeline([
                    ('complaint_n_selector', ItemSelector(key='Жалобы (ngramm)')),
                    ('tfidf', TfidfVectorizer(ngram_range=(1,1), min_df=1, stop_words=stopwords.words('russian')))
                ])),
                ('doctor_pipe', Pipeline([
                    ('doctor_selector', ItemSelector(key='Врач')),
                    ('doc_pop', DoctorsPopularityTransformator()),
                    ('count_vect', CountVectorizer())
                ])),
                ('gender_pipe', Pipeline([
                    ('gender_selector', ItemSelector(key='Пол')),
                    ('gender_transform', GenderTransformator())
                ])),
                ('age_pipe', Pipeline([
                    ('age_selector', ItemSelector(key='Возраст')),
                    ('age_transformator', AgeTransformator())
                ])),
                ('topics_pipe', Pipeline([
                    ('topic_selector', ItemSelector(key=['topic' + str(i) for i in range(355)])),
                    ('topics_transform', TopicsTransformator())
                ])),
                ('clinic_pipe', Pipeline([
                    ('clinic_selector', ItemSelector(key='Клиника')),
                    ('clinic_transform', ClinicTransformator())
                ])),
                ('repeats_pipe', Pipeline([
                    ('repeats_selector', ItemSelector(key='Повторный приём')),
                    ('repeats_transform', RepeatsTransformator())
                ]))
            ]
        )),
        ('clf', NNModel(batch_size=128, epochs=50, tb_log_dir=tb_log_dir,
                        model_path=model_path, validation_split=validation_split))
    ])
    
    return pipe, model_name

In [94]:
train = utils.load_data('data/train_data_complaints_repeats_doctors.csv')
train_topics = np.load('data/topics_train_ngramm.npy')
train = utils.join_topics(train, train_topics)
doctors = train['Врач'].fillna('sss')
doctors_voc, counts = np.unique(doctors, return_counts=True)
pop_doctor = doctors_voc[np.argsort(counts)[::-1][0]]
doctors[doctors == 'sss'] = pop_doctor
train['Врач'] = doctors
train.iloc[np.where(train['Врач'] == 'ЛФК невролог')[0][0], train.columns.get_loc('Врач')] = 'невролог'
# train, valid = train_test_split(train, test_size=0.3)

In [95]:
test = utils.load_data('data/test_data_complaints_repeats_doctors.csv')
test_topics = np.load('data/topics_test_ngramm.npy')
test = utils.join_topics(test, test_topics)
doctors = test['Врач'].fillna('sss')
doctors_voc, counts = np.unique(doctors, return_counts=True)
pop_doctor = doctors_voc[np.argsort(counts)[::-1][0]]
doctors[doctors == 'sss'] = pop_doctor
test['Врач'] = doctors
test.iloc[np.where(test['Врач'] == 'ЛФК невролог')[0][0], test.columns.get_loc('Врач')] = 'невролог'

In [96]:
data = train
doctors_all = np.unique(data['Врач'])

In [97]:
np.unique(data['Врач'], return_counts=True)

(array(['акушер гинеколог', 'аллерголог иммунолог', 'ангиохирург хирург',
        'врачебный', 'гастроэнтеролог', 'гинеколог', 'дерматовенеролог',
        'дерматовенеролог трихолог', 'кардиолог', 'колопроктолог',
        'маммолог онколог', 'невролог', 'невролог травматолог',
        'нейрохирург хирург', 'ортопед', 'ортопед травматолог',
        'оториноларинголог', 'офтальмолог', 'педиатр',
        'психотерапевт терапевт', 'пульмонолог', 'реабилитолог ревматолог',
        'ревматолог', 'рефлексотерапевт терапевт', 'стоматолог',
        'терапевт', 'уролог', 'флеболог хирург', 'хирург', 'эндокринолог'],
       dtype=object),
 array([11242,   513,     3,     9,  4208,     4,  4064,    72,  2459,
          129,   979, 12636,    92,   194,     5,  2808,  4794,  2154,
          106,    74,   121,    52,   388,   246,    22,  6291,  3711,
          472,  1641,  2487]))

In [98]:
np.unique(train[(train['Пол'] != 1) & (train['Пол'] != 2)])

array([], dtype=object)

In [99]:
experiment_dir = 'nn_doctors'
test_submits = []
model_names = dict()

for doctor in doctors_all:
    train_doctor = train[train['Врач'] == doctor]
    pipe, model_name = create_pipeline(experiment_dir, 'nn_2dense', no_validation=True, validation_split=0.3)
    
    train_doctor_y, pop_diagnoses, most_pop_diagnose = preproc_target_train(train_doctor, reduce_classes=False)
    
    pipe.fit(train_doctor, train_doctor_y)
    
    model_names[doctor] = model_name
    with open(join(experiment_dir, 'model_names.json'), 'w') as f:
        json.dump(model_names, f, ensure_ascii=False)
    
    test_doctor = test[test['Врач'] == doctor]
    test_pred = pipe.transform(test_doctor)
    
    test_doctor_submit = pd.DataFrame({'Id_Записи': test_doctor['Id_Записи'], 'Код_диагноза': test_pred})
    test_submits.append(test_doctor_submit)
    
test_submit = pd.concat(test_submits)

/root/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


Epoch 1/50
11242/11242 [==============================] - 1s 80us/step - loss: 3.8789 - acc: 0.1399
Epoch 2/50
11242/11242 [==============================] - 1s 61us/step - loss: 3.3984 - acc: 0.1986
Epoch 3/50
11242/11242 [==============================] - 1s 55us/step - loss: 3.3443 - acc: 0.2252
Epoch 4/50
11242/11242 [==============================] - 1s 56us/step - loss: 3.2929 - acc: 0.2374
Epoch 5/50
11242/11242 [==============================] - 1s 55us/step - loss: 3.2445 - acc: 0.2461
Epoch 6/50
11242/11242 [==============================] - 1s 63us/step - loss: 3.1981 - acc: 0.2511
Epoch 7/50
11242/11242 [==============================] - 1s 51us/step - loss: 3.1532 - acc: 0.2582
Epoch 8/50
11242/11242 [==============================] - 1s 57us/step - loss: 3.1089 - acc: 0.2624
Epoch 9/50
11242/11242 [==============================] - 1s 58us/step - loss: 3.0681 - acc: 0.2685
Epoch 10/50
11242/11242 [==============================] - 1s 57us/step - loss: 3.0303 - acc: 0.2693

/root/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


Epoch 1/50
513/513 [==============================] - 0s 331us/step - loss: 4.3686 - acc: 0.0175
Epoch 2/50
513/513 [==============================] - 0s 43us/step - loss: 3.9466 - acc: 0.0526
Epoch 3/50
513/513 [==============================] - 0s 43us/step - loss: 3.7122 - acc: 0.0663
Epoch 4/50
513/513 [==============================] - 0s 45us/step - loss: 3.5376 - acc: 0.0897
Epoch 5/50
513/513 [==============================] - 0s 45us/step - loss: 3.3718 - acc: 0.2612
Epoch 6/50
513/513 [==============================] - 0s 63us/step - loss: 3.2583 - acc: 0.2612
Epoch 7/50
513/513 [==============================] - 0s 57us/step - loss: 3.1951 - acc: 0.2612
Epoch 8/50
513/513 [==============================] - 0s 65us/step - loss: 3.1449 - acc: 0.2612
Epoch 9/50
513/513 [==============================] - 0s 64us/step - loss: 3.1080 - acc: 0.2612
Epoch 10/50
513/513 [==============================] - 0s 61us/step - loss: 3.0793 - acc: 0.2710
Epoch 11/50
513/513 [=================

/root/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


...

Epoch 1/50
472/472 [==============================] - 0s 590us/step - loss: 4.0170 - acc: 0.0021
Epoch 2/50
472/472 [==============================] - 0s 55us/step - loss: 3.4015 - acc: 0.0593
Epoch 3/50
472/472 [==============================] - 0s 55us/step - loss: 3.0116 - acc: 0.5784
Epoch 4/50
472/472 [==============================] - 0s 58us/step - loss: 2.7133 - acc: 0.5911
Epoch 5/50
472/472 [==============================] - 0s 55us/step - loss: 2.4901 - acc: 0.5911
Epoch 6/50
472/472 [==============================] - 0s 63us/step - loss: 2.3189 - acc: 0.5911
Epoch 7/50
472/472 [==============================] - 0s 56us/step - loss: 2.1976 - acc: 0.5911
Epoch 8/50
472/472 [==============================] - 0s 60us/step - loss: 2.1133 - acc: 0.5911
Epoch 9/50
472/472 [==============================] - 0s 64us/step - loss: 2.0568 - acc: 0.5911
Epoch 10/50
472/472 [==============================] - 0s 58us/step - loss: 2.0201 - acc: 0.5911
Epoch 11/50
472/472 [=================

/root/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


Epoch 1/50
1641/1641 [==============================] - 0s 178us/step - loss: 5.8056 - acc: 0.0079
Epoch 2/50
1641/1641 [==============================] - 0s 52us/step - loss: 5.3743 - acc: 0.0317
Epoch 3/50
1641/1641 [==============================] - 0s 58us/step - loss: 5.2031 - acc: 0.0573
Epoch 4/50
1641/1641 [==============================] - 0s 59us/step - loss: 5.1267 - acc: 0.0555
Epoch 5/50
1641/1641 [==============================] - 0s 58us/step - loss: 5.0926 - acc: 0.0603
Epoch 6/50
1641/1641 [==============================] - 0s 52us/step - loss: 5.0735 - acc: 0.0536
Epoch 7/50
1641/1641 [==============================] - 0s 53us/step - loss: 5.0602 - acc: 0.0591
Epoch 8/50
1641/1641 [==============================] - 0s 59us/step - loss: 5.0484 - acc: 0.0628
Epoch 9/50
1641/1641 [==============================] - 0s 57us/step - loss: 5.0405 - acc: 0.0652
Epoch 10/50
1641/1641 [==============================] - 0s 59us/step - loss: 5.0314 - acc: 0.0615
Epoch 11/50
1641/1

/root/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


Epoch 1/50
2487/2487 [==============================] - 0s 129us/step - loss: 4.0782 - acc: 0.0869
Epoch 2/50
2487/2487 [==============================] - 0s 54us/step - loss: 3.3253 - acc: 0.1122
Epoch 3/50
2487/2487 [==============================] - 0s 55us/step - loss: 3.1194 - acc: 0.1870
Epoch 4/50
2487/2487 [==============================] - 0s 58us/step - loss: 3.0736 - acc: 0.1878
Epoch 5/50
2487/2487 [==============================] - 0s 54us/step - loss: 3.0488 - acc: 0.1998
Epoch 6/50
2487/2487 [==============================] - 0s 58us/step - loss: 3.0288 - acc: 0.2079
Epoch 7/50
2487/2487 [==============================] - 0s 56us/step - loss: 3.0163 - acc: 0.2159
Epoch 8/50
2487/2487 [==============================] - 0s 48us/step - loss: 3.0025 - acc: 0.2139
Epoch 9/50
2487/2487 [==============================] - 0s 54us/step - loss: 2.9915 - acc: 0.2203
Epoch 10/50
2487/2487 [==============================] - 0s 57us/step - loss: 2.9815 - acc: 0.2272
Epoch 11/50
2487/2

In [109]:
test_submit = pd.concat(test_submits)

In [111]:
np.unique(test_submit['Id_Записи']).shape

(29997,)

In [120]:
test_submit = pd.concat([
    test_submit,
    pd.DataFrame({
        'Id_Записи': test[(test['Врач'] == 'ЛФК') | (test['Врач'] == 'ЛФК ортопед')]['Id_Записи'],
        'Код_диагноза': scipy.stats.mode(train['Код_диагноза'])[0][0]
    })
])

/root/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [115]:
np.unique(test['Врач'], return_counts=True)

(array(['ЛФК', 'ЛФК ортопед', 'акушер гинеколог', 'аллерголог иммунолог',
        'ангиохирург хирург', 'врачебный', 'гастроэнтеролог', 'гинеколог',
        'дерматовенеролог', 'дерматовенеролог трихолог', 'кардиолог',
        'колопроктолог', 'маммолог онколог', 'невролог',
        'невролог травматолог', 'нейрохирург хирург', 'ортопед',
        'ортопед травматолог', 'оториноларинголог', 'офтальмолог',
        'педиатр', 'психотерапевт терапевт', 'пульмонолог',
        'реабилитолог ревматолог', 'ревматолог',
        'рефлексотерапевт терапевт', 'стоматолог', 'терапевт', 'уролог',
        'флеболог хирург', 'хирург', 'эндокринолог'], dtype=object),
 array([   2,    1, 5557,  205,    2,    4, 2064,    1, 1967,   31, 1124,
          62,  455, 6236,   35,   98,    6, 1210, 2360, 1052,   44,   34,
          69,   10,  159,  120,   12, 3079, 1861,  248,  709, 1183]))

In [121]:
test_submit.to_csv('submit/bow_simple_nn_doctors_3dense_diag_all_compl_uni-n_gram_doctor_topics_clinic_repeats.csv', header=True, index=False)

In [120]:
train_y, pop_diagnoses, most_pop_diagnose = preproc_target_train(train, reduce_classes=False)
# valid_y, _, _ = preproc_target_test(valid, pop_diagnoses, most_pop_diagnose)

/root/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [121]:
np.unique(train_y).shape

(2302,)

In [ ]:
pipe.fit(train, train_y)

Epoch 1/200
61976/61976 [==============================] - 10s 160us/step - loss: 5.8049 - acc: 0.0519
Epoch 2/200
45056/61976 [====================>.........] - ETA: 2s - loss: 5.5170 - acc: 0.0823

In [18]:
pred = pipe.transform(train)

In [19]:
np.unique(pred, return_counts=True)

(array(['A46', 'A63.0', 'B00', 'B00.1', 'B00.8', 'B02', 'B07', 'B08.1',
        'B35.1', 'B35.3', 'B35.6', 'B36.0', 'B37', 'B37.3+', 'B97.1',
        'B97.7', 'D17.0', 'D17.1', 'D17.2', 'D18.0', 'D23', 'D23.3',
        'D23.5', 'D23.9', 'D24', 'D25', 'D50', 'D50.9', 'D89.8', 'E01.1',
        'E01.8', 'E03.8', 'E03.9', 'E04.1', 'E04.2', 'E05.0', 'E06.3',
        'E11', 'E22.1', 'E28', 'E28.1', 'E28.8', 'E28.9', 'E66.0', 'E89.0',
        'F41.0', 'F41.2', 'F45.3', 'F95.0', 'G24', 'G24.9', 'G43.0',
        'G44.2', 'G50.0', 'G51.0', 'G55*', 'G55.1*', 'G55.3*', 'G56.0',
        'G56.2', 'G90', 'G90.8', 'G90.9', 'G93.4', 'H00', 'H00.0', 'H00.1',
        'H01.0', 'H04.1', 'H10.0', 'H10.1', 'H10.2', 'H10.3', 'H10.5',
        'H10.8', 'H11.3', 'H15.1', 'H16.1', 'H16.2', 'H20.0', 'H35.0',
        'H35.3', 'H35.4', 'H40.1', 'H52.0', 'H52.1', 'H52.2', 'H52.4',
        'H52.5', 'H60', 'H61.2', 'H65', 'H65.0', 'H65.1', 'H66.0', 'H66.1',
        'H68.0', 'H68.1', 'H81.1', 'H81.2', 'H90.3', 'H93.1', 

## Submit

In [9]:
train = utils.load_data('data/train_data_complaints_repeats_doctors.csv')
train_topics = np.load('data/topics_train_ngramm.npy')
train = join_topics(train, train_topics)
train_y, pop_diagnoses, most_pop_diagnose = preproc_target_train(train, reduce_classes=False)

/root/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [10]:
pipe.fit(train, train_y)

Epoch 1/100
61976/61976 [==============================] - 13s 215us/step - loss: 5.8733 - acc: 0.0998
Epoch 2/100
61976/61976 [==============================] - 13s 212us/step - loss: 5.5176 - acc: 0.1808
Epoch 3/100
61976/61976 [==============================] - 13s 211us/step - loss: 5.1080 - acc: 0.2237
Epoch 4/100
61976/61976 [==============================] - 13s 212us/step - loss: 4.7338 - acc: 0.2444
Epoch 5/100
61976/61976 [==============================] - 13s 211us/step - loss: 4.4498 - acc: 0.2577
Epoch 6/100
61976/61976 [==============================] - 13s 211us/step - loss: 4.2481 - acc: 0.2679
Epoch 7/100
61976/61976 [==============================] - 13s 209us/step - loss: 4.1090 - acc: 0.2773
Epoch 8/100
61976/61976 [==============================] - 13s 209us/step - loss: 4.0043 - acc: 0.2860
Epoch 9/100
61976/61976 [==============================] - 13s 209us/step - loss: 3.9136 - acc: 0.2952
Epoch 10/100
61976/61976 [==============================] - 13s 211us/ste

Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('complaints_pipe', Pipeline(memory=None,
     steps=[('complaint_selector', ItemSelector(key='Жалобы (unigramm)')), ('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, e...:48_2018/{epoch:02d}_{acc:.2f}.h5',
    tb_log_dir='simple_models/log/nn14', validation_split=0.0))])

In [113]:
test = utils.load_data('data/test_data_complaints_repeats_doctors.csv')
test_topics = np.load('data/topics_test_ngramm.npy')
test = join_topics(test, test_topics)

In [114]:
test_pred = pipe.transform(test)

In [115]:
submit = pd.DataFrame({'Id_Записи': test['Id_Записи'], 'Код_диагноза': test_pred})

In [116]:
submit['Код_диагноза'].value_counts()

M42.1      2346
J06.9      1734
N76.0      1504
Z32.1      1201
N77.1*     1022
N41.1       950
J00         778
M51         735
I11         722
K29.9       584
M65         575
K30         523
J35.0       520
Z01.8       505
Z00.0       498
M54.8       480
N60.1       427
M53.8       403
D23.9       398
H52.1       378
Z01.4       355
M54.2       354
J01.0       330
K29.5       317
L30.8       289
D25         264
I83.9       261
E04.2       245
G90         225
J04.1       223
           ... 
H81.2         2
Z10           2
J02.9         2
M54.1         2
Z00.1         2
E22.1         2
L91.0         2
H00.1         1
I87.2         1
M43.9         1
J35.3         1
H40.1         1
L08.8         1
L01.0         1
S91.3         1
N94.9         1
M17.0         1
G43.0         1
I49           1
B00.1         1
M65-M68       1
K58.9         1
H65.1         1
L01           1
E05.0         1
N84.1         1
I86.1         1
L04.0         1
G44.1         1
K81           1
Name: Код_диагноза, Leng

In [117]:
submit.to_csv('submit/bow_simple_nn33_3dense_diag_all_compl_uni-n_gram_doctor_topics_clinic_repeats.csv', header=True, index=False)